## RFC Generator using RNNs

Reference: https://github.com/martin-gorner/tensorflow-rnn-shakespeare

In [27]:
import tensorflow as tf
import numpy as np
import os
import glob
import my_txtutils as txt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(tf.__version__)

1.3.0


In [3]:
# Read the RFC docs and generate a list of all characters, encode them

def process_rfc(dirpath, verbose=False):
    characters = list()
    
    for filename in glob.glob(os.path.join(dirpath, '*.txt')):
        if verbose:
            print('Reading filename', filename)
        l = open(filename, 'r', encoding='utf8').read()
        characters.extend(l)
    if verbose:
        print('Total length', len(characters))
        
    return characters

def encode(chars):
    characters = list()
    for c in chars:
        characters.append(ord(c))
    return characters

In [4]:
chars = encode(process_rfc('datasets/rfc'))
charset = list(set(chars))

In [5]:
# Generate training and validation data
train_pct = 95.0
sep = int(np.ceil((len(chars) * train_pct / float(100))))
chars_train = chars[:sep]
chars_valid = chars[sep:]

print('Training set length', len(chars_train))
print('Validation set length', len(chars_valid))

Training set length 25032430
Validation set length 1317496


In [6]:
################### CUSTOM
#chars, valitext, bookranges = txt.read_data_files('datasets/rfc/*.txt', validation=False)
#chars_train = chars
#charset = list(set(chars))
################### CUSTOM

In [7]:
VOCABSIZE = len(charset) # Our vocabulary size
SEQLEN = 30 # We will use a sequence size of 30 steps
BATCHSIZE = 500
NUM_NEURONS = 512
NUM_LAYERS = 3
LEARNING_RATE = 0.001
DROPOUT_PKEEP = 0.75
NUM_EPOCH = len(chars_train) // (BATCHSIZE * SEQLEN)
print('Num epochs', NUM_EPOCH)

Num epochs 1668


In [8]:
################### CUSTOM
#VOCABSIZE = txt.ALPHASIZE
################### CUSTOM

In [9]:
# Place holders
x = tf.placeholder(tf.uint8, [None, None]) # [BATCHSIZE, SEQLEN], SEQLEN will be deduced from the input
x_ohe = tf.one_hot(x, VOCABSIZE, 1.0, 0.0)
y_actual = tf.placeholder(tf.uint8, [None, None]) # We are expecting the same input seq shifted by 1
y_actual_ohe = tf.one_hot(y_actual, VOCABSIZE, 1.0, 0.0)

h_initial_state = tf.placeholder(tf.float32, [None, NUM_NEURONS*NUM_LAYERS])

lr = tf.placeholder(tf.float32)
pkeep = tf.placeholder(tf.float32)
batchsize = tf.placeholder(tf.int32)

In [10]:
# The model, with dropouts

cells = [tf.contrib.rnn.GRUCell(NUM_NEURONS) for _ in range(NUM_LAYERS)]
dropcells = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=pkeep) for cell in cells]
multicell = tf.contrib.rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = tf.contrib.rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)

Yrnn, H = tf.nn.dynamic_rnn(multicell, x_ohe, dtype=tf.float32, initial_state=h_initial_state)
H = tf.identity(H, name='H')
# Shape of Yr is [BATCHSIZE, SEQLEN, NUM_NEURONS]
# Shape of H is [BATCHSIZE, SEQLEN * NUM_NEURONS]

print(cells[0].state_size)
print(multicell.state_size)
print(Yrnn.shape)
print(H.shape)

Yflat = tf.reshape(Yrnn, [-1, NUM_NEURONS]) # [BATCHSIZE * SEQLEN, NUM_NEURONS]

# Bring the actual and prediction to same shape, to compare
# This is our prediction
y_logits = tf.contrib.layers.linear(Yflat, VOCABSIZE) # [BATCHSIZE * SEQLEN, VOCABSIZE]
# This is actual/expected
y_actual_ohe_flat = tf.reshape(y_actual_ohe, [-1, VOCABSIZE]) # [BATCHSIZE * SEQLEN, VOCABSIZE]

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_logits, labels=y_actual_ohe_flat)
xentropy = tf.reshape(xentropy, [batchsize, -1]) # [BATCHSIZE, SEQLEN]

y_output_softmax = tf.nn.softmax(y_logits) # [BATCHSIZE * SEQLEN, VOCABSIZE]
y_output = tf.argmax(y_output_softmax, 1) # [BATCHSIZE * SEQLEN]
y_output = tf.reshape(y_output, [batchsize, -1]) # [BATCHSIZE, SEQLEN]

trainner = tf.train.AdamOptimizer(lr).minimize(xentropy)

512
1536
(?, ?, 512)
(?, 1536)


In [11]:
# To monitor progress

seqloss = tf.reduce_mean(xentropy, 1)
batchloss = tf.reduce_mean(seqloss)

equal = tf.equal(y_actual, tf.cast(y_output, tf.uint8))
accuracy = tf.reduce_mean(tf.cast(equal, tf.float32))

In [12]:
# Checkpoint location
if os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
saver = tf.train.Saver(max_to_keep=1000)

In [13]:

for x_, y_, epoch in txt.rnn_minibatch_sequencer(chars_train, BATCHSIZE, SEQLEN, 1):
    for a in x_:
        print(len(a), end='')
        for b in a:
            print(chr(b), end='')
        print('?', end='')
        break
        
    print('---------------------')
    for a in y_:
        print(len(a), end='')
        for b in a:
            print(chr(b), end='')
        print('?', end='')
        break
        
    break


30
Network Working Group
Request?---------------------
30Network Working Group
Request ?

In [14]:
# Now the training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_idx = 0
hinit = np.zeros([BATCHSIZE, NUM_NEURONS*NUM_LAYERS])

for x_, y_, epoch in txt.rnn_minibatch_sequencer(chars_train, BATCHSIZE, SEQLEN, 2):
    
    feed_dict = {x: x_, 
                 y_actual: y_, 
                 lr: LEARNING_RATE, 
                 pkeep: DROPOUT_PKEEP, 
                 h_initial_state: hinit, 
                 batchsize:BATCHSIZE}
    _, _y, _ostate = sess.run([trainner, y_output, H], feed_dict=feed_dict)
    hinit = _ostate
    
    print('Epoch', epoch, 'Iteration', batch_idx)
    batch_idx += 1

Epoch 0 Iteration 0
Epoch 0 Iteration 1
Epoch 0 Iteration 2
Epoch 0 Iteration 3
Epoch 0 Iteration 4
Epoch 0 Iteration 5
Epoch 0 Iteration 6
Epoch 0 Iteration 7
Epoch 0 Iteration 8
Epoch 0 Iteration 9
Epoch 0 Iteration 10
Epoch 0 Iteration 11
Epoch 0 Iteration 12
Epoch 0 Iteration 13
Epoch 0 Iteration 14
Epoch 0 Iteration 15
Epoch 0 Iteration 16
Epoch 0 Iteration 17
Epoch 0 Iteration 18
Epoch 0 Iteration 19
Epoch 0 Iteration 20
Epoch 0 Iteration 21
Epoch 0 Iteration 22
Epoch 0 Iteration 23
Epoch 0 Iteration 24
Epoch 0 Iteration 25
Epoch 0 Iteration 26
Epoch 0 Iteration 27
Epoch 0 Iteration 28
Epoch 0 Iteration 29
Epoch 0 Iteration 30
Epoch 0 Iteration 31
Epoch 0 Iteration 32
Epoch 0 Iteration 33
Epoch 0 Iteration 34
Epoch 0 Iteration 35
Epoch 0 Iteration 36
Epoch 0 Iteration 37
Epoch 0 Iteration 38
Epoch 0 Iteration 39
Epoch 0 Iteration 40
Epoch 0 Iteration 41
Epoch 0 Iteration 42
Epoch 0 Iteration 43
Epoch 0 Iteration 44
Epoch 0 Iteration 45
Epoch 0 Iteration 46
Epoch 0 Iteration 47
Ep

Epoch 0 Iteration 378
Epoch 0 Iteration 379
Epoch 0 Iteration 380
Epoch 0 Iteration 381
Epoch 0 Iteration 382
Epoch 0 Iteration 383
Epoch 0 Iteration 384
Epoch 0 Iteration 385
Epoch 0 Iteration 386
Epoch 0 Iteration 387
Epoch 0 Iteration 388
Epoch 0 Iteration 389
Epoch 0 Iteration 390
Epoch 0 Iteration 391
Epoch 0 Iteration 392
Epoch 0 Iteration 393
Epoch 0 Iteration 394
Epoch 0 Iteration 395
Epoch 0 Iteration 396
Epoch 0 Iteration 397
Epoch 0 Iteration 398
Epoch 0 Iteration 399
Epoch 0 Iteration 400
Epoch 0 Iteration 401
Epoch 0 Iteration 402
Epoch 0 Iteration 403
Epoch 0 Iteration 404
Epoch 0 Iteration 405
Epoch 0 Iteration 406
Epoch 0 Iteration 407
Epoch 0 Iteration 408
Epoch 0 Iteration 409
Epoch 0 Iteration 410
Epoch 0 Iteration 411
Epoch 0 Iteration 412
Epoch 0 Iteration 413
Epoch 0 Iteration 414
Epoch 0 Iteration 415
Epoch 0 Iteration 416
Epoch 0 Iteration 417
Epoch 0 Iteration 418
Epoch 0 Iteration 419
Epoch 0 Iteration 420
Epoch 0 Iteration 421
Epoch 0 Iteration 422
Epoch 0 It

Epoch 0 Iteration 752
Epoch 0 Iteration 753
Epoch 0 Iteration 754
Epoch 0 Iteration 755
Epoch 0 Iteration 756
Epoch 0 Iteration 757
Epoch 0 Iteration 758
Epoch 0 Iteration 759
Epoch 0 Iteration 760
Epoch 0 Iteration 761
Epoch 0 Iteration 762
Epoch 0 Iteration 763
Epoch 0 Iteration 764
Epoch 0 Iteration 765
Epoch 0 Iteration 766
Epoch 0 Iteration 767
Epoch 0 Iteration 768
Epoch 0 Iteration 769
Epoch 0 Iteration 770
Epoch 0 Iteration 771
Epoch 0 Iteration 772
Epoch 0 Iteration 773
Epoch 0 Iteration 774
Epoch 0 Iteration 775
Epoch 0 Iteration 776
Epoch 0 Iteration 777
Epoch 0 Iteration 778
Epoch 0 Iteration 779
Epoch 0 Iteration 780
Epoch 0 Iteration 781
Epoch 0 Iteration 782
Epoch 0 Iteration 783
Epoch 0 Iteration 784
Epoch 0 Iteration 785
Epoch 0 Iteration 786
Epoch 0 Iteration 787
Epoch 0 Iteration 788
Epoch 0 Iteration 789
Epoch 0 Iteration 790
Epoch 0 Iteration 791
Epoch 0 Iteration 792
Epoch 0 Iteration 793
Epoch 0 Iteration 794
Epoch 0 Iteration 795
Epoch 0 Iteration 796
Epoch 0 It

Epoch 0 Iteration 1120
Epoch 0 Iteration 1121
Epoch 0 Iteration 1122
Epoch 0 Iteration 1123
Epoch 0 Iteration 1124
Epoch 0 Iteration 1125
Epoch 0 Iteration 1126
Epoch 0 Iteration 1127
Epoch 0 Iteration 1128
Epoch 0 Iteration 1129
Epoch 0 Iteration 1130
Epoch 0 Iteration 1131
Epoch 0 Iteration 1132
Epoch 0 Iteration 1133
Epoch 0 Iteration 1134
Epoch 0 Iteration 1135
Epoch 0 Iteration 1136
Epoch 0 Iteration 1137
Epoch 0 Iteration 1138
Epoch 0 Iteration 1139
Epoch 0 Iteration 1140
Epoch 0 Iteration 1141
Epoch 0 Iteration 1142
Epoch 0 Iteration 1143
Epoch 0 Iteration 1144
Epoch 0 Iteration 1145
Epoch 0 Iteration 1146
Epoch 0 Iteration 1147
Epoch 0 Iteration 1148
Epoch 0 Iteration 1149
Epoch 0 Iteration 1150
Epoch 0 Iteration 1151
Epoch 0 Iteration 1152
Epoch 0 Iteration 1153
Epoch 0 Iteration 1154
Epoch 0 Iteration 1155
Epoch 0 Iteration 1156
Epoch 0 Iteration 1157
Epoch 0 Iteration 1158
Epoch 0 Iteration 1159
Epoch 0 Iteration 1160
Epoch 0 Iteration 1161
Epoch 0 Iteration 1162
Epoch 0 Ite

Epoch 0 Iteration 1478
Epoch 0 Iteration 1479
Epoch 0 Iteration 1480
Epoch 0 Iteration 1481
Epoch 0 Iteration 1482
Epoch 0 Iteration 1483
Epoch 0 Iteration 1484
Epoch 0 Iteration 1485
Epoch 0 Iteration 1486
Epoch 0 Iteration 1487
Epoch 0 Iteration 1488
Epoch 0 Iteration 1489
Epoch 0 Iteration 1490
Epoch 0 Iteration 1491
Epoch 0 Iteration 1492
Epoch 0 Iteration 1493
Epoch 0 Iteration 1494
Epoch 0 Iteration 1495
Epoch 0 Iteration 1496
Epoch 0 Iteration 1497
Epoch 0 Iteration 1498
Epoch 0 Iteration 1499
Epoch 0 Iteration 1500
Epoch 0 Iteration 1501
Epoch 0 Iteration 1502
Epoch 0 Iteration 1503
Epoch 0 Iteration 1504
Epoch 0 Iteration 1505
Epoch 0 Iteration 1506
Epoch 0 Iteration 1507
Epoch 0 Iteration 1508
Epoch 0 Iteration 1509
Epoch 0 Iteration 1510
Epoch 0 Iteration 1511
Epoch 0 Iteration 1512
Epoch 0 Iteration 1513
Epoch 0 Iteration 1514
Epoch 0 Iteration 1515
Epoch 0 Iteration 1516
Epoch 0 Iteration 1517
Epoch 0 Iteration 1518
Epoch 0 Iteration 1519
Epoch 0 Iteration 1520
Epoch 0 Ite

Epoch 1 Iteration 1836
Epoch 1 Iteration 1837
Epoch 1 Iteration 1838
Epoch 1 Iteration 1839
Epoch 1 Iteration 1840
Epoch 1 Iteration 1841
Epoch 1 Iteration 1842
Epoch 1 Iteration 1843
Epoch 1 Iteration 1844
Epoch 1 Iteration 1845
Epoch 1 Iteration 1846
Epoch 1 Iteration 1847
Epoch 1 Iteration 1848
Epoch 1 Iteration 1849
Epoch 1 Iteration 1850
Epoch 1 Iteration 1851
Epoch 1 Iteration 1852
Epoch 1 Iteration 1853
Epoch 1 Iteration 1854
Epoch 1 Iteration 1855
Epoch 1 Iteration 1856
Epoch 1 Iteration 1857
Epoch 1 Iteration 1858
Epoch 1 Iteration 1859
Epoch 1 Iteration 1860
Epoch 1 Iteration 1861
Epoch 1 Iteration 1862
Epoch 1 Iteration 1863
Epoch 1 Iteration 1864
Epoch 1 Iteration 1865
Epoch 1 Iteration 1866
Epoch 1 Iteration 1867
Epoch 1 Iteration 1868
Epoch 1 Iteration 1869
Epoch 1 Iteration 1870
Epoch 1 Iteration 1871
Epoch 1 Iteration 1872
Epoch 1 Iteration 1873
Epoch 1 Iteration 1874
Epoch 1 Iteration 1875
Epoch 1 Iteration 1876
Epoch 1 Iteration 1877
Epoch 1 Iteration 1878
Epoch 1 Ite

Epoch 1 Iteration 2194
Epoch 1 Iteration 2195
Epoch 1 Iteration 2196
Epoch 1 Iteration 2197
Epoch 1 Iteration 2198
Epoch 1 Iteration 2199
Epoch 1 Iteration 2200
Epoch 1 Iteration 2201
Epoch 1 Iteration 2202
Epoch 1 Iteration 2203
Epoch 1 Iteration 2204
Epoch 1 Iteration 2205
Epoch 1 Iteration 2206
Epoch 1 Iteration 2207
Epoch 1 Iteration 2208
Epoch 1 Iteration 2209
Epoch 1 Iteration 2210
Epoch 1 Iteration 2211
Epoch 1 Iteration 2212
Epoch 1 Iteration 2213
Epoch 1 Iteration 2214
Epoch 1 Iteration 2215
Epoch 1 Iteration 2216
Epoch 1 Iteration 2217
Epoch 1 Iteration 2218
Epoch 1 Iteration 2219
Epoch 1 Iteration 2220
Epoch 1 Iteration 2221
Epoch 1 Iteration 2222
Epoch 1 Iteration 2223
Epoch 1 Iteration 2224
Epoch 1 Iteration 2225
Epoch 1 Iteration 2226
Epoch 1 Iteration 2227
Epoch 1 Iteration 2228
Epoch 1 Iteration 2229
Epoch 1 Iteration 2230
Epoch 1 Iteration 2231
Epoch 1 Iteration 2232
Epoch 1 Iteration 2233
Epoch 1 Iteration 2234
Epoch 1 Iteration 2235
Epoch 1 Iteration 2236
Epoch 1 Ite

Epoch 1 Iteration 2552
Epoch 1 Iteration 2553
Epoch 1 Iteration 2554
Epoch 1 Iteration 2555
Epoch 1 Iteration 2556
Epoch 1 Iteration 2557
Epoch 1 Iteration 2558
Epoch 1 Iteration 2559
Epoch 1 Iteration 2560
Epoch 1 Iteration 2561
Epoch 1 Iteration 2562
Epoch 1 Iteration 2563
Epoch 1 Iteration 2564
Epoch 1 Iteration 2565
Epoch 1 Iteration 2566
Epoch 1 Iteration 2567
Epoch 1 Iteration 2568
Epoch 1 Iteration 2569
Epoch 1 Iteration 2570
Epoch 1 Iteration 2571
Epoch 1 Iteration 2572
Epoch 1 Iteration 2573
Epoch 1 Iteration 2574
Epoch 1 Iteration 2575
Epoch 1 Iteration 2576
Epoch 1 Iteration 2577
Epoch 1 Iteration 2578
Epoch 1 Iteration 2579
Epoch 1 Iteration 2580
Epoch 1 Iteration 2581
Epoch 1 Iteration 2582
Epoch 1 Iteration 2583
Epoch 1 Iteration 2584
Epoch 1 Iteration 2585
Epoch 1 Iteration 2586
Epoch 1 Iteration 2587
Epoch 1 Iteration 2588
Epoch 1 Iteration 2589
Epoch 1 Iteration 2590
Epoch 1 Iteration 2591
Epoch 1 Iteration 2592
Epoch 1 Iteration 2593
Epoch 1 Iteration 2594
Epoch 1 Ite

Epoch 1 Iteration 2910
Epoch 1 Iteration 2911
Epoch 1 Iteration 2912
Epoch 1 Iteration 2913
Epoch 1 Iteration 2914
Epoch 1 Iteration 2915
Epoch 1 Iteration 2916
Epoch 1 Iteration 2917
Epoch 1 Iteration 2918
Epoch 1 Iteration 2919
Epoch 1 Iteration 2920
Epoch 1 Iteration 2921
Epoch 1 Iteration 2922
Epoch 1 Iteration 2923
Epoch 1 Iteration 2924
Epoch 1 Iteration 2925
Epoch 1 Iteration 2926
Epoch 1 Iteration 2927
Epoch 1 Iteration 2928
Epoch 1 Iteration 2929
Epoch 1 Iteration 2930
Epoch 1 Iteration 2931
Epoch 1 Iteration 2932
Epoch 1 Iteration 2933
Epoch 1 Iteration 2934
Epoch 1 Iteration 2935
Epoch 1 Iteration 2936
Epoch 1 Iteration 2937
Epoch 1 Iteration 2938
Epoch 1 Iteration 2939
Epoch 1 Iteration 2940
Epoch 1 Iteration 2941
Epoch 1 Iteration 2942
Epoch 1 Iteration 2943
Epoch 1 Iteration 2944
Epoch 1 Iteration 2945
Epoch 1 Iteration 2946
Epoch 1 Iteration 2947
Epoch 1 Iteration 2948
Epoch 1 Iteration 2949
Epoch 1 Iteration 2950
Epoch 1 Iteration 2951
Epoch 1 Iteration 2952
Epoch 1 Ite

Epoch 1 Iteration 3268
Epoch 1 Iteration 3269
Epoch 1 Iteration 3270
Epoch 1 Iteration 3271
Epoch 1 Iteration 3272
Epoch 1 Iteration 3273
Epoch 1 Iteration 3274
Epoch 1 Iteration 3275
Epoch 1 Iteration 3276
Epoch 1 Iteration 3277
Epoch 1 Iteration 3278
Epoch 1 Iteration 3279
Epoch 1 Iteration 3280
Epoch 1 Iteration 3281
Epoch 1 Iteration 3282
Epoch 1 Iteration 3283
Epoch 1 Iteration 3284
Epoch 1 Iteration 3285
Epoch 1 Iteration 3286
Epoch 1 Iteration 3287
Epoch 1 Iteration 3288
Epoch 1 Iteration 3289
Epoch 1 Iteration 3290
Epoch 1 Iteration 3291
Epoch 1 Iteration 3292
Epoch 1 Iteration 3293
Epoch 1 Iteration 3294
Epoch 1 Iteration 3295
Epoch 1 Iteration 3296
Epoch 1 Iteration 3297
Epoch 1 Iteration 3298
Epoch 1 Iteration 3299
Epoch 1 Iteration 3300
Epoch 1 Iteration 3301
Epoch 1 Iteration 3302
Epoch 1 Iteration 3303
Epoch 1 Iteration 3304
Epoch 1 Iteration 3305
Epoch 1 Iteration 3306
Epoch 1 Iteration 3307
Epoch 1 Iteration 3308
Epoch 1 Iteration 3309
Epoch 1 Iteration 3310
Epoch 1 Ite

In [16]:
VOCABSIZE

100

In [28]:
import my_txtutils as txt1

In [29]:
txt.print_text_generation_header()
ry = np.array([[txt.convert_from_alphabet(ord("b"))]])
rh = np.zeros([1, NUM_NEURONS * NUM_LAYERS])
for k in range(2000):
    ryo, rh = sess.run([y_output_softmax, H], feed_dict={x: ry, pkeep: 1.0, h_initial_state: rh, batchsize: 1})
    #print(ryo.shape)
    rc = txt1.sample_from_probabilities(ryo, 10, VOCABSIZE)
    print(chr(txt.convert_to_alphabet(rc)), end="")
    ry = np.array([[rc]])
txt.print_text_generation_footer()


┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
rjcrgLgLLLLLLLLLLLLLLLLLLLLLLLLL>>>>>>>>>>K 
>>>((>>>(>>>>L>>>>>>
K
 >pJr
>r>>>J> 
>>>J>>>>>>>>>>>
>>>>(>>>> >(>>K>>>>>>>>>>>>K>>>>>K>(>>>>(>>>>>>>>>>K>>>>> >>>>(>>>(>>>(>>>>>(>>>>>>>>>>>J
r 
>>>>>>>>>L>>>>>>>>>>>Lg>>


>>L>>>r>>L>>r>
>K>((>>>>>>>>>>>>>L>
>(>>> >>>>>>>>>r>>>>
> >>>
>(>>>>>>>>>> >   rK>>> >
>>
>
>>>>p>(>>>>L>>l
>((((((((((((((((((>>>>>>>>>>>>>>>>>>>>>>>>n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>yK

>>N{(*(pda>OSUP>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>(>>>>>>>>>>>>>>>>>>(>>>>>>>>>>>>>> >>>>>_>>>>>>>>>>>>>>>>
L>>>>>>>>>>_>(>>>>>>>
L >>L>>>>>
>>_J r>>>
>>>>_>>(>>>(>>>>>> >J
>p>
>>>
>>>>>>>KK
 >>
 >>>> >>K(>>> >>>>
> L>r
> >>>
L
>>>>>>>q
>>L>L>> >>
(>>>>>>>q(>qL>J>> > >>pr>  K>>> >> >>>>>>(>>>>
>>LL>
 >K>J >>>>>>>> >J>>>>>> >>>>>>>
>>>>>>>
>(>>>>>>> >>>>>
>>p>>>>>


>
>> 
>J>>L>>>>>>>>>>>>> p>>>>
>>>>>>>>>>pL((>>>>>>>>>>>>>K>

>>K>>>L
 >K